In [1]:
!python -m pip install pandas requests


[notice] A new release of pip available: 22.3 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import json
import pandas as pd
import requests

In [3]:
def curl_json(url):
    return json.loads(requests.get(url).content)

versions = curl_json("https://ddragon.leagueoflegends.com/api/versions.json")
last_version = versions[0]
champion = curl_json(f"http://ddragon.leagueoflegends.com/cdn/{last_version}/data/en_US/champion.json")
names = [v["name"] for k,v in champion["data"].items()]
print(f"version: {last_version}\n{len(names)} champions")

version: 13.6.1
163 champions


In [4]:
def shortcuts_generator(s):
    ls = len(s)
    for i in range(1, 2 ** len(s)):
        mask = bin(i)[2:].rjust(ls, '0')
        shortcut = "".join([si for i, si in enumerate(s) if mask[i] == "1"])
        yield shortcut


names_shortcuts = [(name, set(shortcuts_generator(name.lower().replace(" ", "")))) for name in names]

shortest_shortcuts = []

for name, shortcuts in names_shortcuts:
    copy = list(names_shortcuts)
    copy.remove((name, shortcuts))
    all_others = set()
    for n, s in copy:
        all_others |= s
    shortcut = list(shortcuts - all_others)
    if not shortcut:
        shortest_shortcuts.append((name, ""))
        continue
    min_len = min([len(s) for s in shortcut])
    shortest_shortcut = {s for s in shortcut if len(s) == min_len}
    shortest_shortcuts.append((name, ", ".join(shortest_shortcut)))

In [8]:
df = pd.DataFrame(shortest_shortcuts, columns=["Champions", "Champion select shortcuts"])
open("README.md", "w").write(df.to_markdown())

33989